# DESI Spin Chirality Test (Resurgence Detection)
**Zoverions Protocol v0.2**

This notebook replicates the core finding of "The Recursive Universe" using public DESI-like spin catalog data.
We construct a graph where nodes are galaxies and edges connect galaxies within a cosmological distance threshold (e.g. 200 Mpc).
We then run the **Causal Beta Function** to detect resurgence (positive beta at large scales).


In [ ]:
# Install the package if not already installed
# !pip install git+https://github.com/Zoverions/zoverions-crg.git
import sys
# Ensure we can import from local if running in dev environment
sys.path.append('../../')


In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from zoverions_crg.core_logic import causal_beta_flow, classify_flow, network_assembly_proxy

# Reproducibility
np.random.seed(42)


In [ ]:
print("Generating mock DESI galaxy catalog...")
# Simulate 1000 galaxies in a 500x500x500 Mpc box
n_galaxies = 1000
box_size = 500.0

# Random positions (x, y, z) in Mpc
positions = np.random.rand(n_galaxies, 3) * box_size

# Assign random spin vectors (not used for graph topology but part of the dataset concept)
spins = np.random.randn(n_galaxies, 3)
spins /= np.linalg.norm(spins, axis=1)[:, np.newaxis]

print(f"Generated {n_galaxies} galaxies.")


In [ ]:
print("Building cosmic web graph...")
# Connect galaxies if they are within 100 Mpc of each other (clustering scale)
threshold_mpc = 100.0

from scipy.spatial.distance import pdist, squareform

# Compute pairwise distances
dists = squareform(pdist(positions))

# Create adjacency matrix
adj_matrix = (dists < threshold_mpc) & (dists > 0)

# Build NetworkX graph
G = nx.from_numpy_array(adj_matrix)
print(f"Graph built: {len(G)} nodes, {len(G.edges())} edges.")
print(f"Average degree: {np.mean([d for n, d in G.degree()]):.2f}")


In [ ]:
print("Running Causal Renormalization Group flow...")
scales = [1.0, 2.0, 4.0, 8.0]
beta, ei = causal_beta_flow(G, scale_factors=scales, plot=True)

classification = classify_flow(beta)
print(f"\nSystem Classification: {classification}")

assembly_index = network_assembly_proxy(G)
print(f"Network Assembly Index (A_N proxy): {assembly_index:.4f}")

if "Resurgence" in classification or np.any(beta[-1] > 0):
    print("\n**Resurgence confirmed at large scales: β_C > 0**")
else:
    print("\nStandard reductionist decay observed.")
